In [1]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath('src'))
from utopia.microservice.generate_object.generate_object_app import *
from utopia.utopia import utopiaModel
import pymongo
import json

In [2]:
# Load the default configuration and data from the 'data' folder.
data_data = utopiaModel.load_json_file("data/default_data.json")
config_data = utopiaModel.load_json_file("data/default_config.json")

In [3]:
# Pretty print with indentation
print("Loaded Configuration Data:")
print(json.dumps(config_data, indent=2))
print("\nLoaded Input Data:")
print(json.dumps(data_data, indent=2))

Loaded Configuration Data:
{
  "big_bin_diameter_um": 5000,
  "N_sizeBins": 5,
  "vol_algal_cell_m3": 2e-16,
  "spm_density_kg_m3": 1388,
  "boxName": "Utopia",
  "comp_input_file_name": "inputs_compartments.csv",
  "comp_interactFile_name": "compartment_interactions.csv",
  "MPforms_list": [
    "freeMP",
    "heterMP",
    "biofMP",
    "heterBiofMP"
  ],
  "solver": "SteadyState",
  "compartment_types": {
    "UTOPIA_surfaceSea_water_compartments": [
      "Ocean_Surface_Water",
      "Coast_Surface_Water"
    ],
    "UTOPIA_water_compartments": [
      "Ocean_Mixed_Water",
      "Ocean_Column_Water",
      "Coast_Column_Water",
      "Surface_Freshwater",
      "Bulk_Freshwater"
    ],
    "UTOPIA_deep_soil_compartments": [
      "Beaches_Deep_Soil",
      "Background_Soil",
      "Impacted_Soil"
    ],
    "UTOPIA_soil_surface_compartments": [
      "Beaches_Soil_Surface",
      "Background_Soil_Surface",
      "Impacted_Soil_Surface"
    ],
    "UTOPIA_sediment_compartment": [
  

### Insert initial data (config_data and data_data) into database 

In [4]:
initialize_mongo_collections()

In [5]:

from datetime import datetime
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['utopia']
config_collection = db['configure_data']
input_collection = db['input_data']
config_document = config_collection.insert_one(config_data)
input_document = input_collection.insert_one(data_data)

In [6]:
# Clear the model_json collection
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['utopia']
model_json_collection = db["model_json"]
model_json_collection.delete_many({}) 

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [7]:
from bson import json_util

model_json = create_model_json()
generate_coding_dictionaries()
print("\nModel JSON:")
print(json_util.dumps(model_json, indent=2))
print("\nType of model_json['particles_df']:", type(model_json["particles_df"]))

Model JSON created and inserted into MongoDB collection 'model_json'.

Model JSON:
{
  "MPdensity_kg_m3": 980,
  "MP_composition": "PE",
  "shape": "sphere",
  "MP_form": "freeMP",
  "big_bin_diameter_um": 5000,
  "N_sizeBins": 5,
  "FI": 0.5,
  "t_half_deg_free": 66000,
  "t_frag_gen_FreeSurfaceWater": 36.5,
  "heter_deg_factor": 10,
  "biof_deg_factor": 0.5,
  "factor_deepWater_soilSurface": 10,
  "factor_sediment": 100,
  "biof_frag_factor": 2,
  "heter_frag_factor": 100,
  "vol_algal_cell_m3": 2e-16,
  "spm_density_kg_m3": 1388,
  "comp_input_file_name": "inputs_compartments.csv",
  "comp_interactFile_name": "compartment_interactions.csv",
  "boxName": "Utopia",
  "MPforms_list": [
    "freeMP",
    "heterMP",
    "biofMP",
    "heterBiofMP"
  ],
  "compartments_list": [
    "Ocean_Surface_Water",
    "Ocean_Mixed_Water",
    "Ocean_Column_Water",
    "Coast_Surface_Water",
    "Coast_Column_Water",
    "Surface_Freshwater",
    "Bulk_Freshwater",
    "Sediment_Freshwater",
    "Se

In [8]:
print(compartment_surfaceSea_water.__module__)
print(compartment_surfaceSea_water.__dict__.keys())


utopia.objects.compartment_classes_json
dict_keys(['__module__', '__init__', 'to_json', 'to_dict', '__doc__'])


In [9]:
(
        system_particle_object_list_json,
        SpeciesList,
        spm_dict,
        dict_comp,
        particles_properties_df_dict
) = generate_objects_json(model_json)



Assigned Ocean_Surface_Water volume: 1630000000000000.0 m3
Assigned Ocean_Mixed_Water volume: 3.26e+16 m3
Assigned Ocean_Column_Water volume: 1.31e+18 m3
Assigned Coast_Surface_Water volume: 67100000000000.0 m3
Assigned Coast_Column_Water volume: 1340000000000000.0 m3
Assigned Surface_Freshwater volume: 9050000000000.0 m3
Assigned Bulk_Freshwater volume: 90500000000000.0 m3
Assigned Sediment_Freshwater volume: 905000000000.0 m3
Assigned Sediment_Ocean volume: 9790000000000.0 m3
Assigned Sediment_Coast volume: 2680000000000.0 m3
Assigned Beaches_Soil_Surface volume: 3440.0 m3
Assigned Beaches_Deep_Soil volume: 34400.0 m3
Assigned Background_Soil_Surface volume: 873000000000.0 m3
Assigned Background_Soil volume: 8730000000000.0 m3
Assigned Impacted_Soil_Surface volume: 607000000000.0 m3
Assigned Impacted_Soil volume: 6070000000000.0 m3
Assigned Air volume: 3.06E+18 m3
modleBoxes:  [{'Bname': 'Utopia', 'Bdepth_m': None, 'Blength_m': None, 'Bwidth_m': None, 'Bvolume_m3': None, 'Bconexions'

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['utopia']
model_json_collection = db["model_json"]

# Update or insert the document with the new data
model_json_collection.find_one_and_update(
	{},  # filter (empty to match any document)
	{
		"$set": {
			"system_particle_object_list": system_particle_object_list_json,
			"SpeciesList": SpeciesList,
			"spm": spm_dict,
			"dict_comp": dict_comp,
			"particles_properties_df": particles_properties_df_dict
		}
	},
	upsert=True
)

{'_id': ObjectId('685821eb1951c8eb47cfeeb8'),
 'MPdensity_kg_m3': 980,
 'MP_composition': 'PE',
 'shape': 'sphere',
 'MP_form': 'freeMP',
 'big_bin_diameter_um': 5000,
 'N_sizeBins': 5,
 'FI': 0.5,
 't_half_deg_free': 66000,
 't_frag_gen_FreeSurfaceWater': 36.5,
 'heter_deg_factor': 10,
 'biof_deg_factor': 0.5,
 'factor_deepWater_soilSurface': 10,
 'factor_sediment': 100,
 'biof_frag_factor': 2,
 'heter_frag_factor': 100,
 'vol_algal_cell_m3': 2e-16,
 'spm_density_kg_m3': 1388,
 'comp_input_file_name': 'inputs_compartments.csv',
 'comp_interactFile_name': 'compartment_interactions.csv',
 'boxName': 'Utopia',
 'MPforms_list': ['freeMP', 'heterMP', 'biofMP', 'heterBiofMP'],
 'compartments_list': ['Ocean_Surface_Water',
  'Ocean_Mixed_Water',
  'Ocean_Column_Water',
  'Coast_Surface_Water',
  'Coast_Column_Water',
  'Surface_Freshwater',
  'Bulk_Freshwater',
  'Sediment_Freshwater',
  'Sediment_Ocean',
  'Sediment_Coast',
  'Beaches_Soil_Surface',
  'Beaches_Deep_Soil',
  'Background_Soil

In [ ]:
model_json_document = model_json_collection.find_one()
for key in model_json_document.keys():
    print(key)

Keys of model_json_document ['_id', 'MPdensity_kg_m3', 'MP_composition', 'shape', 'MP_form', 'big_bin_diameter_um', 'N_sizeBins', 'FI', 't_half_deg_free', 't_frag_gen_FreeSurfaceWater', 'heter_deg_factor', 'biof_deg_factor', 'factor_deepWater_soilSurface', 'factor_sediment', 'biof_frag_factor', 'heter_frag_factor', 'vol_algal_cell_m3', 'spm_density_kg_m3', 'comp_input_file_name', 'comp_interactFile_name', 'boxName', 'MPforms_list', 'compartments_list', 'solver', 'compartment_types', 'radius_algae_m', 'spm_radius_um', 'emiss_dict_g_s', 'particles_df', 'base_path', 'MP_form_dict_reverse', 'comp_dict_inverse', 'dict_size_coding', 'particle_compartmentCoding', 'particle_forms_coding', 'size_dict', 'SpeciesList', 'dict_comp', 'particles_properties_df', 'spm', 'system_particle_object_list']


In [10]:
print("\nSystem Particle Object List JSON:")
print(json.dumps(system_particle_object_list_json, indent=2))


System Particle Object List JSON:
[
  {
    "Pcomposition": "PE",
    "Pdensity_kg_m3": 980.0,
    "PdimensionX_m": 2.5e-07,
    "PdimensionX_um": 0.25,
    "PdimensionY_m": 2.5e-07,
    "PdimensionY_um": 0.25,
    "PdimensionZ_m": 2.5e-07,
    "PdimensionZ_um": 0.25,
    "Pemiss_t_y": 0,
    "Pform": "freeMP",
    "Pname": "mp1",
    "Pnumber_t0": null,
    "Pshape": "sphere",
    "diameter_m": 5e-07,
    "diameter_um": 0.5,
    "radius_m": 2.5e-07,
    "t_half_d": 5000,
    "Pvolume_m3": 6.544984694978735e-20,
    "CSF": 1,
    "Pcompartment_Cname": "Ocean_Surface_Water",
    "Pcompartment_CBox_Bname": "Utopia",
    "Pcode": "aA0_Utopia"
  },
  {
    "Pcomposition": "PE",
    "Pdensity_kg_m3": 980.0,
    "PdimensionX_m": 2.5e-06,
    "PdimensionX_um": 2.5,
    "PdimensionY_m": 2.5e-06,
    "PdimensionY_um": 2.5,
    "PdimensionZ_m": 2.5e-06,
    "PdimensionZ_um": 2.5,
    "Pemiss_t_y": 0,
    "Pform": "freeMP",
    "Pname": "mp2",
    "Pnumber_t0": null,
    "Pshape": "sphere",
    

In [11]:
print("\nSpecies List:")
print(json.dumps(SpeciesList, indent=2))



Species List:
[
  "aA0_Utopia",
  "bA0_Utopia",
  "cA0_Utopia",
  "dA0_Utopia",
  "eA0_Utopia",
  "aB0_Utopia",
  "bB0_Utopia",
  "cB0_Utopia",
  "dB0_Utopia",
  "eB0_Utopia",
  "aC0_Utopia",
  "bC0_Utopia",
  "cC0_Utopia",
  "dC0_Utopia",
  "eC0_Utopia",
  "aD0_Utopia",
  "bD0_Utopia",
  "cD0_Utopia",
  "dD0_Utopia",
  "eD0_Utopia",
  "aA1_Utopia",
  "bA1_Utopia",
  "cA1_Utopia",
  "dA1_Utopia",
  "eA1_Utopia",
  "aB1_Utopia",
  "bB1_Utopia",
  "cB1_Utopia",
  "dB1_Utopia",
  "eB1_Utopia",
  "aC1_Utopia",
  "bC1_Utopia",
  "cC1_Utopia",
  "dC1_Utopia",
  "eC1_Utopia",
  "aD1_Utopia",
  "bD1_Utopia",
  "cD1_Utopia",
  "dD1_Utopia",
  "eD1_Utopia",
  "aA2_Utopia",
  "bA2_Utopia",
  "cA2_Utopia",
  "dA2_Utopia",
  "eA2_Utopia",
  "aB2_Utopia",
  "bB2_Utopia",
  "cB2_Utopia",
  "dB2_Utopia",
  "eB2_Utopia",
  "aC2_Utopia",
  "bC2_Utopia",
  "cC2_Utopia",
  "dC2_Utopia",
  "eC2_Utopia",
  "aD2_Utopia",
  "bD2_Utopia",
  "cD2_Utopia",
  "dD2_Utopia",
  "eD2_Utopia",
  "aA3_Utopia",
  "bA3_

In [12]:
print("\nSPM Dictionary:")
print(json.dumps(spm_dict, indent=2))



SPM Dictionary:
{
  "Pcomposition": "Mixed",
  "Pdensity_kg_m3": 1388,
  "PdimensionX_m": 3.6278316785978123e-06,
  "PdimensionX_um": 3.627831678597812,
  "PdimensionY_m": 0.0,
  "PdimensionY_um": 0,
  "PdimensionZ_m": 0.0,
  "PdimensionZ_um": 0,
  "Pemiss_t_y": 0,
  "Pform": "suspendedParticulates",
  "Pname": "spm1",
  "Pnumber_t0": null,
  "Pshape": "sphere",
  "diameter_m": 7.255663357195625e-06,
  "diameter_um": 7.255663357195624,
  "radius_m": 3.6278316785978123e-06,
  "t_half_d": 5000,
  "Pvolume_m3": 2.0000000000000044e-16,
  "CSF": 1
}


In [13]:
print("\nComposition Dictionary:")
print(json.dumps(dict_comp, indent=2))



Composition Dictionary:
{
  "Ocean_Surface_Water": {
    "Cname": "Ocean_Surface_Water",
    "Cdepth_m": 5.0,
    "Clength_m": null,
    "Cwidth_m": null,
    "Cvolume_m3": 1630000000000000.0,
    "CsurfaceArea_m2": 326000000000000.0,
    "particles": {
      "freeMP": [],
      "heterMP": [],
      "biofMP": [],
      "heterBiofMP": []
    },
    "processess": [
      "discorporation",
      "fragmentation",
      "heteroaggregation",
      "heteroaggregate_breackup",
      "biofouling",
      "defouling",
      "advective_transport",
      "settling",
      "rising",
      "mixing",
      "sea_spray_aerosol",
      "beaching"
    ],
    "connexions": {
      "Ocean_Mixed_Water": [
        "settling",
        "mixing"
      ],
      "Coast_Surface_Water": "advective_transport",
      "Air": "sea_spray_aerosol"
    },
    "SPM_mgL": 1.0,
    "flowVelocity_m_s": 0.03,
    "waterFlow_m3_s": 67100000.0,
    "T_K": 278.0,
    "G": 1.0,
    "compartment_type": "surfaceSea_water",
    "Ccod

In [14]:
print("\nParticles Properties DataFrame Dictionary:")
print(json.dumps(particles_properties_df_dict, indent=2))


Particles Properties DataFrame Dictionary:
[
  {
    "Particle": "mp1",
    "Radius_m": 2.5e-07,
    "Volume_m3": 6.544984694978735e-20,
    "Density_kg_m3": 980.0,
    "Corey Shape Factor": 1
  },
  {
    "Particle": "mp2",
    "Radius_m": 2.5e-06,
    "Volume_m3": 6.544984694978737e-17,
    "Density_kg_m3": 980.0,
    "Corey Shape Factor": 1
  },
  {
    "Particle": "mp3",
    "Radius_m": 2.5e-05,
    "Volume_m3": 6.544984694978736e-14,
    "Density_kg_m3": 980.0,
    "Corey Shape Factor": 1
  },
  {
    "Particle": "mp4",
    "Radius_m": 0.00025,
    "Volume_m3": 6.544984694978736e-11,
    "Density_kg_m3": 980.0,
    "Corey Shape Factor": 1
  },
  {
    "Particle": "mp5",
    "Radius_m": 0.0025,
    "Volume_m3": 6.544984694978737e-08,
    "Density_kg_m3": 980.0,
    "Corey Shape Factor": 1
  },
  {
    "Particle": "mp1_BF",
    "Radius_m": 3.8778316785978124e-06,
    "Volume_m3": 1.771152082578197e-15,
    "Density_kg_m3": 1387.8906765504773,
    "Corey Shape Factor": 1
  },
  {
  

In [16]:
print("\nModel JSON:")
print(json_util.dumps(model_json, indent=2))



Model JSON:
{
  "MPdensity_kg_m3": 980,
  "MP_composition": "PE",
  "shape": "sphere",
  "MP_form": "freeMP",
  "big_bin_diameter_um": 5000,
  "N_sizeBins": 5,
  "FI": 0.5,
  "t_half_deg_free": 66000,
  "t_frag_gen_FreeSurfaceWater": 36.5,
  "heter_deg_factor": 10,
  "biof_deg_factor": 0.5,
  "factor_deepWater_soilSurface": 10,
  "factor_sediment": 100,
  "biof_frag_factor": 2,
  "heter_frag_factor": 100,
  "vol_algal_cell_m3": 2e-16,
  "spm_density_kg_m3": 1388,
  "comp_input_file_name": "inputs_compartments.csv",
  "comp_interactFile_name": "compartment_interactions.csv",
  "boxName": "Utopia",
  "MPforms_list": [
    "freeMP",
    "heterMP",
    "biofMP",
    "heterBiofMP"
  ],
  "compartments_list": [
    "Ocean_Surface_Water",
    "Ocean_Mixed_Water",
    "Ocean_Column_Water",
    "Coast_Surface_Water",
    "Coast_Column_Water",
    "Surface_Freshwater",
    "Bulk_Freshwater",
    "Sediment_Freshwater",
    "Sediment_Ocean",
    "Sediment_Coast",
    "Beaches_Soil_Surface",
    "